In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import gmaps
import gmaps.geojson_geometries

from config import gkey

crime_df = pd.read_csv('data/crime_by_state.csv')
crime_df.head()

,year,state_abbr,state_id,population,aggravated_assault,arson,burglary,homicide,larceny,motor_vehicle_theft,property_crime,rape_legacy,rape_revised,robbery,violent_crime
0,1983,AL,2,3959000,11281,NaN,42485,364,94279,9126,145890,931,NaN,3895,16471
1,1984,AL,2,3990000,11995,NaN,39970,374,89198,9319,138487,1002,NaN,3833,17204
2,2001,AL,2,4468912,12250,NaN,40642,379,119992,12619,173253,1369,NaN,5584,19582
3,2002,AL,2,4478896,12002,NaN,42578,303,123932,13890,180400,1664,NaN,5962,19931
4,2003,AL,2,4503726,11338,NaN,43245,299,124039,14957,182241,1656,NaN,6038,19331


In [2]:
crime2017_df = crime_df[crime_df['year'] == 2017]
crime2017_df.head()

,year,state_abbr,state_id,population,aggravated_assault,arson,burglary,homicide,larceny,motor_vehicle_theft,property_crime,rape_legacy,rape_revised,robbery,violent_crime
17,2017,AL,2,4874747,18902,703.0,31477,404,99842,12841,144160,0,2028.0,4217,25551
56,2017,AK,1,739795,4257,139.0,4171,62,17775,4258,26204,0,863.0,951,6133
103,2017,AZ,5,7016270,24207,1183.0,37627,416,147830,19058,204515,0,3581.0,7440,35644
119,2017,AR,3,3004279,12425,534.0,21862,258,63374,7253,92489,0,2053.0,1935,16671
161,2017,CA,6,39536653,104454,8648.0,176690,1830,642033,168391,987114,0,14721.0,56622,177627


In [3]:
crime2017_df = crime2017_df[['state_abbr', 'violent_crime', 'property_crime', 'population']]
crime2017_df['violent_crime_per100'] = crime2017_df['violent_crime']*100/crime2017_df['population']
crime2017_df['property_crime_per100'] = crime2017_df['property_crime']*100/crime2017_df['population']
crime2017_df['total_crime_per100'] = crime2017_df['violent_crime_per100']+crime2017_df['property_crime_per100']
crime2017_df.head()

,state_abbr,violent_crime,property_crime,population,violent_crime_per100,property_crime_per100,total_crime_per100
17,AL,25551,144160,4874747,0.524150,2.957282,3.481432
56,AK,6133,26204,739795,0.829013,3.542062,4.371076
103,AZ,35644,204515,7016270,0.508019,2.914868,3.422887
119,AR,16671,92489,3004279,0.554909,3.078576,3.633484
161,CA,177627,987114,39536653,0.449272,2.496706,2.945978


In [4]:
crime2017_sorted= crime2017_df.sort_values(by = ['total_crime_per100'], ascending = False)
crime2017_sorted.head()

,state_abbr,violent_crime,property_crime,population,violent_crime_per100,property_crime_per100,total_crime_per100
1171,NM,16359,82306,2088070,0.783451,3.941726,4.725177
56,AK,6133,26204,739795,0.829013,3.542062,4.371076
682,LA,26092,157712,4684333,0.557006,3.366797,3.923803
1522,SC,25432,160575,5024369,0.506173,3.195924,3.702097
119,AR,16671,92489,3004279,0.554909,3.078576,3.633484


In [5]:
max_crime = max(crime2017_df['total_crime_per100'])
crime_val = crime2017_df['total_crime_per100']/max(crime2017_df['total_crime_per100'])
crime_val = crime_val.tolist()

In [6]:
norm = colors.Normalize(vmin=min(crime_val), vmax = max(crime_val))

m = cm.ScalarMappable(norm = norm, cmap='OrRd')
colormap = m.to_rgba(crime_val)
clr = [colors.to_hex(c) for c in colormap]

colorcoded_states = pd.Series(clr, index = crime2017_df['state_abbr'])
colorcoded_states = colorcoded_states.to_frame()

state_abbr = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN",
              "IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV",
              "NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN",
              "TX","UT","VT","VA","WA","WV","WI","WY"]

states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

state_names = pd.Series(states, index = state_abbr)
state_names = state_names.to_frame()

state_color = colorcoded_states.merge(state_names, right_index = True, left_index = True)
state_color = state_color.rename(index=str, columns={"0_x": "color", "0_y": "state"})
state_color.head()

,color,state
state_abbr,,
AL,#f16c4b,Alabama
AK,#ad0000,Alaska
AZ,#f3724e,Arizona
AR,#e9583e,Arkansas
CA,#fda56f,California


In [7]:
states_geo = gmaps.geojson_geometries.load_geometry('us-states')
st = [i['properties']['NAME'] for i in states_geo['features']]
missing = [s for s in st if s not in states]

missing_df = pd.DataFrame([['#ffffff', missing[0]], ['#ffffff', missing[1]]], columns=list(state_color), index = ['DC', 'PR'])
state_color = state_color.append(missing_df)
state_order = pd.DataFrame(pd.Series(st), columns = ['state'])

df = state_order.merge(state_color, on = 'state')

In [8]:
df.head()

,state,color
0,Arizona,#f3724e
1,Arkansas,#e9583e
2,California,#fda56f
3,Colorado,#fc9662
4,Connecticut,#fee7c7


In [9]:
gmaps.configure(api_key = gkey)
fig = gmaps.figure()

states_layer = gmaps.geojson_layer(states_geo, 
                                   fill_color = list(df['color']))
fig.add_layer(states_layer)
fig

Figure(layout=FigureLayout(height='420px'))